Version 2.0.0 of SWTLoc was a major release, consisting of the following major changes

- Addition of new engine for Stroke Width Transform - `numba`, which gives more than 10x speed increase
- Addition of `saveCrops` function
- Re-Organisation of codebase to use Abstractions such as classes for
    - Letter : Class representing a Letter
    - Word : Class representing a Word
    - SWTImage : Class representing an SWT Image
- Consolidation of various localisation methods for ``Letter`` and ``Word`` objects
- Accessing intermediary stage images using Image Codes

# Imports

In [ ]:
# Check version runing on Jupyter notebook
from platform import python_version
import sys

print('Python Version : ', python_version())
print('Python Path : ', sys.executable)

In [ ]:
import sys
sys.path.append('../')

In [ ]:
%load_ext autoreload
%autoreload 2
from swtloc import SWTLocalizer
from swtloc._utils import imgshowN
from swtloc.configs import get_code_descriptions, CODE_NAME_VAR_MAPPINGS

In [ ]:
import os
import pandas as pd
import numpy as np
from IPython.display import clear_output

# Path and Variable Initialisations

In [ ]:
imagefolder_path = 'images/'

In [ ]:
img_paths = []
res_path = []
img_names = []
img_text_modes = ['db_lf', 'lb_df', 'db_lf', 'lb_df', 'db_lf', 'db_lf']

for each_img in [k for k in os.listdir(imagefolder_path) if 'test' in k]:
    _ifolder_path = imagefolder_path+each_img
    _iname = [k for k in os.listdir(_ifolder_path) if '.' in k][0]
    _img_path = _ifolder_path+'/'+_iname
    img_paths.append(_img_path)
    img_names.append(_iname)
    res_path.append(_ifolder_path+'/imp_results/')

In [ ]:
swtl = SWTLocalizer(image_paths=img_paths)
print(swtl.swtimages)

# Speed Benchmarking

From v2.0.0 onwards, there were changes made to the core algorithm implementations of finding stroke widths. A new engine `numba` was added as a parameter to the `SWTImage.transformImage` which shows (initial testing) nearly 50x speed improvement over the vanilla python implementation of the algorithm. Below code block performs benchmarking for the above statement


In [ ]:
_cols = ['SWTLoc v1.1.1 (Python)', 'SWTLoc v2.0.0 (Python)', 'SWTLoc v2.0.0 (numba)']
timedf = pd.DataFrame(index=img_names, columns=_cols)

for img_name, each_img_path, _text_mode in zip(img_names, img_paths, img_text_modes):
    
    swtl =SWTLocalizer(image_paths=each_img_path)
    swtlImgObj = swtl.swtimages[0]
    _t1 = []
    _t2 = []
    _t3 = []
    
    # Python - Old version timing
    for _ in range(10):
        swtl.swttransform(imgpaths=each_img_path, text_mode=_text_mode)
        _t1.append(float(swtl.transform_time.split(' ')[0]))
    
    imgshowN([swtl.orig_img, swtl.swt_mat, swtl.swtlabelled_pruned13C],
         ['Original Image', 'Stroke Width Transform', 'Connected Components'])
    timedf.loc[img_name, _cols[0]] = np.mean(_t1)

    # Python - New version timing
    for _ in range(10):
        swtlImgObj.transformImage(text_mode=_text_mode, engine='python', display=False)
        _t2.append(float(swtlImgObj.transform_time.split(' ')[0]))

    swtlImgObj.transformImage(text_mode=_text_mode, engine='python', display=True)
    timedf.loc[img_name, _cols[1]] = np.mean(_t2)

    # numba - New version timing
    for _ in range(10):
        swtlImgObj.transformImage(text_mode=_text_mode, display=False)
        _t3.append(float(swtlImgObj.transform_time.split(' ')[0]))

    swtlImgObj.transformImage(text_mode=_text_mode, display=True)
    timedf.loc[img_name, _cols[2]] = np.mean(_t3)
    
    
    clear_output(wait=True)
    # input()

In [ ]:
timedf

In [ ]:
time_comp_df = timedf.copy()

_old_py_timings = time_comp_df[_cols[0]]
_new_py_timings = time_comp_df[_cols[1]]
_new_nb_timings = time_comp_df[_cols[2]]
time_comp_df = time_comp_df.applymap(lambda x: str(round(x, 3))+' seconds')
time_comp_df[_cols[0]+'[x Increase]'] = (_old_py_timings/_old_py_timings).apply(round, args=(3,)).astype(str) + 'x'
time_comp_df[_cols[1]+'[x Increase]'] = (_old_py_timings/_new_py_timings).apply(round, args=(3,)).astype(str) + 'x'
time_comp_df[_cols[2]+'[x Increase]'] = (_old_py_timings/_new_nb_timings).apply(round, args=(3,)).astype(str) + 'x'
time_comp_df = time_comp_df[sorted(time_comp_df.columns)]
time_comp_df.style.set_properties(**{'width': '110px', 'text-align': 'center'})


# Save intermediary stage images via Image Codes

From v2.0.0 onwards, a provision was added to save the intermediary stage images by access the `SWTImage.showImage` function. 

<div class="alert alert-block alert-info">
<b>NOTE:</b> 
<u>SWTImage</u> class procedures are sequential in nature, so for example if Letters havent yet been localized then there will be an error pointing to the fact that a particular function needs to be run before that Image Codes [IMAGE_PRUNED_3C_LETTER_LOCALIZATIONS, IMAGE_ORIGINAL_LETTER_LOCALIZATIONS, IMAGE_ORIGINAL_MASKED_LETTER_LOCALIZATIONS] are made available.
</div>

**Available Image Codes**

In [ ]:
image_code_df = pd.DataFrame(columns=['Description'])

for each_code_name, each_code in CODE_NAME_VAR_MAPPINGS.items():
    image_code_df.loc[each_code_name] = get_code_descriptions(each_code).replace('\n', ' ')
image_code_df.style.set_properties(**{'width': '600px', 'text-align': 'center'})


**Transforming Image**

In [ ]:
swtl = SWTLocalizer(image_paths=img_paths)
print(swtl.swtimages)

In [ ]:
swtImgObj = swtl.swtimages[0]
swtImgObj.showImage()

In [ ]:
swt_mat = swtImgObj.transformImage(text_mode='db_lf', maximum_angle_deviation=np.pi/2,
                                   edge_function='ac', gaussian_blurr_kernel=(11, 11),
                                   minimum_stroke_width=5, maximum_stroke_width=50, display=False)
localized_letters = swtImgObj.localizeLetters(display=False)
localized_words = swtImgObj.localizeWords(display=False)

**Display Multiple Intermediary Stage Images**

In [ ]:
# Import the Image Codes from configs
from swtloc.configs import (IMAGE_SWT_TRANSFORMED,
                            IMAGE_CONNECTED_COMPONENTS_3C_WITH_PRUNED_ELEMENTS)

In [ ]:
swtImgObj.showImage(image_codes=[IMAGE_SWT_TRANSFORMED, 
                                 IMAGE_CONNECTED_COMPONENTS_3C_WITH_PRUNED_ELEMENTS],
                    plot_title='SWT Image and Components which were pruned')

**Display Single Intermediary Stage Image**

In [ ]:
swtImgObj.showImage(image_codes=[IMAGE_SWT_TRANSFORMED],
                    plot_title='SWT Image')

**Save Multiple Intermediary images in a single plot**

In [ ]:
swtImgObj.showImage(image_codes=[IMAGE_SWT_TRANSFORMED, 
                                 IMAGE_CONNECTED_COMPONENTS_3C_WITH_PRUNED_ELEMENTS],
                    plot_title='SWT Image and Components which were pruned', 
                    save_fig=True, save_dir=res_path[0])

**Save single Intermediary images in a single plot**

In [ ]:
swtImgObj.showImage(image_codes=[IMAGE_SWT_TRANSFORMED],
                    plot_title='SWT Image', 
                    save_fig=True, save_dir=res_path[0])

# Localization annotation for Letters and Words

From v2.0.0 onwards, two classes `Letter` and `Word` were incuclated representing the abstraction for a letter and a word.
There are various methods for the annotation of each localization annotation

For `Letter`'s, the localization annotations available are: 

    - "min_bbox" : Minimum Bounding Box Boundary Localization Annotation
    - "ext_bbox" : External Bounding Box Boundary Localization Annotation
    - "outline" : Contour Boundary Localization Annotation
    
For `Word`'s, the localization annotations available are: 

    - "bubble" : Fused Bubble Boundary Localization Annotation
    - "bbox" : Fused Bounding Box Boundary Localization Annotation
    - "polygon" : Contour Boundary Localization Annotation

In [ ]:
swtImgObj = swtl.swtimages[1]
swtImgObj.showImage()

In [ ]:
swt_mat = swtImgObj.transformImage(maximum_angle_deviation=np.pi/2,
                                   edge_function='ac', gaussian_blurr_kernel=(11, 11),
                                   minimum_stroke_width=5, maximum_stroke_width=50, display=False)

In [ ]:
localized_letters = swtImgObj.localizeLetters()
localized_letters = swtImgObj.localizeLetters(localize_by='ext_bbox')
localized_letters = swtImgObj.localizeLetters(localize_by='outline')

In [ ]:
localized_words = swtImgObj.localizeWords()
localized_words = swtImgObj.localizeWords(localize_by='bbox')
localized_words = swtImgObj.localizeWords(localize_by='polygon')

# Generating Crops

From v2.0.0 onwards, provision to save a crop of the `Letter` or a `Word` has been added via the function `SWTImage.saveCrop`. The crops can be made on any one of the available image codes (See [Available Image Codes & Their Meanings](#Save-intermediary-stage-images-via-Image-Codes)) for a particular `letter` or `word` key

In [ ]:
swtImgObj = swtl.swtimages[2]
swtImgObj.showImage()

In [ ]:
swt_mat = swtImgObj.transformImage(text_mode='db_lf', maximum_angle_deviation=np.pi/2,
                                   edge_function='ac', gaussian_blurr_kernel=(11, 11),
                                   minimum_stroke_width=5, maximum_stroke_width=50, display=True)
localized_letters = swtImgObj.localizeLetters(display=False)
localized_words = swtImgObj.localizeWords(localize_by='polygon', display=False)

**Saving Crops of Letters**

To assist with visualising letter of interest, whose crop need to be saved, there is a function `SWTImage.getLetter` which visualizes a particular queried letter by its label.

In [ ]:
localized_letters

In [ ]:
_ = swtImgObj.getLetter(3)

In [ ]:
# To save the crop of 3rd letter as shown in the 
from swtloc.configs import (IMAGE_ORIGINAL,
                            IMAGE_SWT_TRANSFORMED)
swtImgObj.saveCrop(save_path=res_path[2], crop_of='letters', crop_key=3,
                   crop_on=IMAGE_SWT_TRANSFORMED, crop_type='min_bbox')
swtImgObj.saveCrop(save_path=res_path[2], crop_of='letters', crop_key=3,
                   crop_on=IMAGE_ORIGINAL, crop_type='min_bbox')

**Saving Crops of Words**


To assist with visualising word of interest, whose crop need to be saved, there is a function `SWTImage.getWord` which visualizes a particular queried word by its label.

In [ ]:
localized_words

In [ ]:
_ = swtImgObj.getWord(11, localize_by='polygon')

In [ ]:
# To save the crop of 3rd letter as shown in the 
from swtloc.configs import (IMAGE_ORIGINAL,
                            IMAGE_SWT_TRANSFORMED)
swtImgObj.saveCrop(save_path=res_path[2], crop_of='words', 
                   crop_key=11, crop_on=IMAGE_SWT_TRANSFORMED, crop_type='polygon')
swtImgObj.saveCrop(save_path=res_path[2], crop_of='words', 
                   crop_key=11, crop_on=IMAGE_ORIGINAL, crop_type='polygon')

# Random Testing Space